In [ ]:
#!/usr/bin/python2
# encoding=utf-8
use_mpi_model = False
import sys,cv2,os,argparse,copy
from img2keypoint_using_ai import parse_heatpaf
from pprint import pprint
sys.path.append("/data1/yks/mxnet_ai/mxnet_pose_for_AI_challenger")
from modelCPMWeight import CPMModel_test
if not use_mpi_model:
    from train import numoflinks,numofparts,save_prefix
    start_epoch = 4700
else:
    save_prefix = "../../models/openpose/realtimePose_mpi"
    start_epoch=0
    numofparts = 16
    numoflinks = 28
import matplotlib.pyplot as plt
%matplotlib inline
def imshow(x,y):
    fig = plt.gcf();fig.set_size_inches(8, 8);plt.title(x); plt.imshow(y);plt.show()
import mxnet as mx
import numpy as np
max_img_shape = (368,368)
imgshape_bind = (max_img_shape,)
def padimg(img,destsize):
    s = img.shape
    print img.shape,destsize/s[1],destsize/s[1]

    if(s[0] > s[1]):
        img_d = cv2.resize(img,dsize = None,fx = 1.0 * destsize/s[0], fy = 1.0 * destsize/s[0])
        img_temp = np.ones(shape = (destsize,destsize,3),dtype=np.uint8) * 128
        sd = img_d.shape
        img_temp[0:sd[0],0:sd[1],0:sd[2]]=img_d
    else:
        img_d = cv2.resize(img,dsize = None,fx = 1.0 * destsize/s[1],fy = 1.0 * destsize/s[1])
        img_temp = np.ones(shape = (destsize,destsize,3),dtype=np.uint8) * 128 
        sd = img_d.shape
        img_temp[0:sd[0],0:sd[1],0:sd[2]]=img_d
    return img_temp
def getHeatAndPAF(img_path,models):
    oriImg = cv2.imread(img_path) # B,G,R order
    oriImg = padimg(oriImg,max_img_shape[0])
    class DataBatch(object):
        def __init__(self, data, label, pad=0):
            self.data = [data]
            self.label = 0
            self.pad = pad
    def preprocess(x_):
        r = []
        for size in imgshape_bind:
            imgs_resize = cv2.resize(x_, (size[0], size[1]), interpolation=cv2.INTER_CUBIC)
            imgs_transpose = np.transpose(np.float32(imgs_resize[:,:,:]), (2,0,1))/256 - 0.5
            imgs_batch = DataBatch(mx.nd.array([imgs_transpose[:,:,:]]), 0)
            r.append(imgs_batch)
        return r
    def suffix_heatmap(heatmap,size):
        heatmap = np.moveaxis(heatmap.asnumpy()[0], 0, -1)
        heatmap = cv2.resize(heatmap, (size[0], size[1]), interpolation=cv2.INTER_CUBIC)            
        return heatmap
    def suffix_paf(paf,size):
        paf = np.moveaxis(paf.asnumpy()[0], 0, -1)
        paf = cv2.resize(paf, (size[0], size[1]), interpolation=cv2.INTER_CUBIC)
        return paf
    
    def _getHeatPAF(args):
        onedata,model = args
        model.forward(onedata)
        result = model.get_outputs()
        max_shape = (oriImg.shape[0],oriImg.shape[1])
        heatmap = suffix_heatmap(result[1],max_shape)
        paf =     suffix_paf(    result[0],max_shape)
        return (heatmap,paf)

    imgs = preprocess(oriImg)
    results = _getHeatPAF((imgs[0],models[0]))
    heatmap_avg,paf_avg = results

    return img_path,oriImg,heatmap_avg,paf_avg
def getModel(prefix,epoch,gpus = [0]):
    sym  = CPMModel_test()
    batch_size = 1
    sym_load, newargs, _ = mx.model.load_checkpoint(prefix, epoch)
    if use_mpi_model:
        sym = sym_load
    model = mx.mod.Module(symbol=sym, context=[mx.gpu(x) for x in gpus],                        
                          label_names=['heatmaplabel',
                                'partaffinityglabel',
                                'heatweight',
                                'vecweight'])
    model.bind(data_shapes=[('data', (batch_size, 3, max_img_shape[0], max_img_shape[1]))],for_training = False)
    model.init_params(arg_params=newargs, aux_params={}, allow_missing=False)
    return model



In [ ]:
def test(imgs_path):
    cmodel = getModel(save_prefix,start_epoch)
    for x,y,z in os.walk(imgs_path):
        for name in z:
            img_path = os.path.join(x,name)
            pprint(img_path)
            img_path,oriImg,heatmap_avg,paf_avg = getHeatAndPAF(img_path,[cmodel])
            imshow("heatmap_avg",np.float32( heatmap_avg[:,:,14]))
            imshow("oriImg",oriImg)
            parse_heatpaf(img_path,oriImg,heatmap_avg,paf_avg,"/tmp/tmp.json",im_show_cb = imshow)
            
#             for i in range(heatmap_avg.shape[2]):
#                 imshow("heatmap_{0}".format(i),heatmap_avg[:,:,i] )
#             for i in range(paf_avg.shape[2]):
#                 paf = np.sqrt(paf_avg[:,:,i * 2].astype(np.float32) ** 2 +
#                               paf_avg[:,:,i *2 +1].astype(np.float32) **2) 
#                 print(paf.shape)
                
#                 imshow("paf_avg_{0}".format(i),paf)



In [ ]:
if __name__ == "__main__":
    #test("/data1/yks/dataset/ai_challenger/ai_challenger_keypoint_validation_20170911/keypoint_validation_images_20170911")
     test('/data1/yks/dataset/openpose_dataset/mpi/images/')   